## Activity inside ROI

Calculates the ROI for an embryo, using the structural channel to calculate the mask.

This code relies on the naming convention in this codebase to work properly: each embryo should have a file ending in `ch1.tif` (active channel) and another one ending in `ch2.tif` (structural channel).

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
from tifffile import imread

from snazzy_processing import activity, find_hatching, roi, utils

dataset_name = '20240611'
root_dir = Path.cwd().parent
dataset_dir = root_dir.joinpath('data', dataset_name)

In [ ]:
img_dir = dataset_dir.joinpath("embs")

# All structural channel movies end with the suffix ch2
active = sorted(img_dir.glob("*ch1.tif"), key=utils.emb_number)
struct = sorted(img_dir.glob("*ch2.tif"), key=utils.emb_number)

### Percentual error caused by downsampling

Percentual error caused by downsampling. 
The error is the percentual difference between the average values for each image slice.

In [ ]:
i = 1
print(f"Using embryo {active[i].stem}")
hp = find_hatching.find_hatching_point(active[i])
# adjust hatching point to match the downsample size
hp -= hp % 50

active_img = imread(active[i], key=range(0, hp))
struct_img = imread(struct[i], key=range(0, hp))


def activity_when_window(window):
    mask = roi.get_roi(struct_img, window=window)
    masked = activity.apply_mask(active_img, mask)
    return activity.get_activity(masked)


def get_error(observed, expected):
    return (observed - expected) / expected


base_activity = activity_when_window(1)

fig, ax = plt.subplots(figsize=(16, 4))

# relative error when sampling in steps of 50, 10, and 5
for i in [50, 10, 5]:
    diff = get_error(activity_when_window(i), base_activity)
    ax.plot(diff[:-100], label=f"{i}:1", alpha=0.7, linewidth=1)

ax.legend()
fig.suptitle("Percentual error for downsampling")
fig.canvas.header_visible = False
fig.canvas.resizable = False
plt.show()

Visualize the active and structural channel signals, sampled in intervals equal to `window`.

In [ ]:
i = 1
window = 10

hp = find_hatching.find_hatching_point(struct[i])
# adjust hatching point to match the downsample size
hp -= hp % window

active_img = imread(active[i], key=range(hp))
struct_img = imread(struct[i], key=range(hp))

mask = roi.get_roi(struct_img, window=window)

masked_active = activity.apply_mask(active_img, mask)
masked_struct = activity.apply_mask(struct_img, mask)

active_signal = activity.get_activity(masked_active)
struct_signal = activity.get_activity(masked_struct)


fig, (ax0, ax1) = plt.subplots(2, figsize=(16, 6))
ax0.plot(active_signal, label="active", color="green")
ax1.plot(struct_signal, label="structural", color="red")
fig.suptitle(active[i].stem)
fig.canvas.header_visible = False
fig.canvas.resizable = False
plt.show()